In [1]:
import datetime
from pandas_datareader import data as pdr
import fix_yahoo_finance as yf
from pandas_datareader import Options
import pandas as pd
from bokeh.plotting import figure, output_file, show,figure
from dateutil import parser

#get data using finance.yahoo.com APIs 
def get_data_from_api():
    #get the historical data 
    yf.pdr_override()
    today=datetime.datetime.today().strftime('%Y-%m-%d')
    data1 = pdr.get_data_yahoo("AMZN", start="2017-10-01", end=today)
    data1.reset_index(inplace=True,drop=False)
    
    #get the options data form Yahoo
    ops = Options('AMZN', 'yahoo')
    data2 = ops.get_all_data()
    expiry = datetime.date.today()
    data2.reset_index(inplace=True,drop=False)
    datatemp = data2.filter(['Last_Trade_Date'], axis=1)
    datatemp['Date'] = [d.date() for d in datatemp['Last_Trade_Date']]
    data2['Date']=pd.to_datetime(datatemp['Date'])
    #return 2 dataframes
    return data1,data2

#Process options data's  Last_Trade_Date column to Date same format as
#in historical data inorder to join these 2 datasets 
def process_data_from_web(historic_data,options):
    datatemp = options_data.filter(['Last_Trade_Date'], axis=1)
    datatemp['Date'] = [parser.parse(d).date() for d in datatemp['Last_Trade_Date']]
    options_data['Date']=pd.to_datetime(datatemp['Date'])
    
    datatemp_h = historic_data.filter(['Date'], axis=1)
    datatemp_h['Date_h'] = [parser.parse(d) for d in datatemp_h['Date']]
    historic_data['Date']=pd.to_datetime(datatemp_h['Date'])
    
#merge two datasets, perform a inner join on data column
def merge_dataframes(data1,data2):
    merged_data=pd.merge(data1, data2, how='inner', left_on='Date', right_on='Date')
    return merged_data

#If close value is greater than open value then the trend is increasing, else decreasing.(used to plot graph)
def inc_dec(close,openv):
    if close>openv:
        value="Increase"
    elif close<openv:
        value="Decrease"
    else:
        value="Equal"
    return value

#Used to get the mid value between close value and open value. (Used to plot the graph)
def get_mid(close,openv):
    return (close+openv)/2

#Since we are generating the graph with x-axis as date, we are removing the duplicates and
# generate new columns middle(mid of open value and close value ),
#status(increase/decrease/equal), height (difference of close and open values of stock)
def preprocess_for_graph_gen(data_frame):
    data_frame=data_frame.drop_duplicates(['Date'], keep='last')
    data_frame['Middle'] =(data_frame['Open']+data_frame['Close'])/2
    data_frame['Status']=data_frame.apply(lambda row: inc_dec(row['Close'],row['Open']), axis=1)
    data_frame['Height']=abs(data_frame['Close']-data_frame['Open'])
    return data_frame

#Generates and prints graph for given dataframe
def generateGraph(df):
    p=figure(x_axis_type='datetime',width=1000,height=300,responsive=True)
    hours_12=12*60*60*1000
    #If the status is increasing we set the color to green, x=date,y=middle
    p.rect(df.Date[df.Status =="Increase"],
           df.Middle[df.Status=="Increase"],
           hours_12,
           df.Height[df.Status=="Increase"],
           fill_color='#00FF7F',
           line_color="black")
    
    #If the status is decreasing we set the color to red, x=date,y=middle
    p.rect(df.Date[df.Status =="Decrease"],
           df.Middle[df.Status=="Decrease"],
           hours_12,
           df.Height[df.Status=="Decrease"],
           fill_color='#DC143C',
           line_color="black")
    #Add a line graph over the bar charts
    p.line(df.Date,df.Close)
    #print(df.head)
    show(p)
    

#If the user enter Yes, the data is fetched from API, if user option is no, 
#data from the hist.csv and opts.csv generated by web scraping using the other 2 scripts is used to generate
#the graph.

api=input("Do you want to fetch the data using API?(Valid options : Yes / No)")
print("No to the option will fetch data by web scraping.")
print(api)
if api=="Yes" or api=="No":
    if api=="Yes":
        #Generate graph on data from APIs
        historic_data,opts_data=get_data_from_api()
        merged_data=merge_dataframes(historic_data,opts_data)
        process_data=preprocess_for_graph_gen(merged_data)
        generateGraph(process_data)
    else:
        print("To process data generated by web scraping, you should have ran Historic_Data_Web_Scraper.ipynb and Options_Data_Web_Scraper.ipynb")
        run_1=input("Have you ran these files?")
        if run_1=="Yes" or run_1=="No":
            if run_1=="Yes":
                #Generate graph for data fetched by web scraping
                historic_data=pd.read_csv("hist.csv")
                options_data=pd.read_csv("opts.csv")
                process_data_from_web(historic_data,options_data)
                merged_data=merge_dataframes(historic_data,options_data)
                process_data=preprocess_for_graph_gen(merged_data)
                generateGraph(process_data)
               
            else:
                print("Please run the above mentioned files, which will generate hist.csv and opts.csv.")
                
        else:
            print("Invalid User Input")
        
    
else:
    print("Invalid User Input")
    

/Users/radhikakalaiselvan/anaconda/lib/python3.6/site-packages/fix_yahoo_finance/__init__.py:43: DeprecationWarning: 
    Auto-overriding of pandas_datareader's get_data_yahoo() is deprecated and no longer available.
    Use pdr_override() to explicitly override it.
  DeprecationWarning)
/Users/radhikakalaiselvan/anaconda/lib/python3.6/site-packages/bokeh/core/json_encoder.py:52: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  NP_EPOCH = np.datetime64('1970-01-01T00:00:00Z')


Do you want to fetch the data using API?(Valid options : Yes / No)Yes
No to the option will fetch data by web scraping.
Yes
[*********************100%***********************]  1 of 1 downloaded

/Users/radhikakalaiselvan/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
